<a href="https://colab.research.google.com/github/kaigouthro/InvokeAI_EasyBook/blob/main/InvokeAI_EasyBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            #@title Check if you have a gpu..
!nvidia-smi -L
!nvidia-smi

# hold a variable to auto skip installs on restart kernel
_run = False

## Sign up/accept terms/get tokens at 
-  https://huggingface.co/runwayml/stable-diffusion-v1-5 
-  https://huggingface.co/runwayml/stable-diffusion-inpainting 
-  https://ngrok.com/  (for access outside)


In [ ]:
#@title # `Enter your info`: { vertical-output: true, form-width: "60%", display-mode: "form" }
from google.colab import drive
import IPython
import os
import sys
from IPython.utils import capture
from IPython.display import *
exists = os.path.exists

#@markdown ### `set to OFF if PERSIST INSTALL already performed`
initializing   =     True #@param {type:"boolean"}
_skip = _run

# Set if instance should be temporary or persistent
#@markdown |   |
#@markdown | - |
#@markdown |`Keep this permanently?`   |
persist_in_drive      = True #@param {type:"boolean"}

#@markdown location default -> `/content/drive/MyDrive/workspace_folder`
# Name of your working folder
workspace_folder = "sd/iv" #@param {type:"string"}

#@markdown  _`tick this = /content/drive/Shareddrives/workkspace_folder`_
shared_drive          = True #@param {type:"boolean"}
_shared = "MyDrive/" 
if shared_drive:
    _shared = 'Shareddrives/' 

# Set Path either way, it can go temp or to drive.
workspace             = '/content/drive/' + _shared + workspace_folder

# Set up storage
if persist_in_drive:  
    # mount if chosen
    drive.mount('/content/drive',force_remount=True)

# create path if not exists (persistent or temp, ssame folder)
os.makedirs(workspace,exist_ok=True)

# Sign up/accept terms/get tokens at https://huggingface.co/runwayml/stable-diffusion-v1-5 https://huggingface.co/runwayml/stable-diffusion-inpainting https://ngrok.com/
huggingface_token = "" #@param {type:"string"}
ngrok_token = "" #@param {type:"string"}



----
# **` Prep -  you can RUN all the rest from here`**
----

In [ ]:
#@title #install the dependencies. :{ vertical-output: false,display-mode: "form" }

if _skip:
    print('installed already')
else:
    %pip install --upgrade pip
    %pip install --upgrade pyngrok
    %pip install virtualenv --force-reinstall
    _run = True

In [ ]:
#@title ## Create InvokeAI root

invokeai_root  = workspace 
os.makedirs ( invokeai_root  , exist_ok = True )
os.chdir    ( invokeai_root  )
print       ( os.getcwd())     


In [ ]:
#@title  One Time Install to storage -> Create virtual environment
if not os.path.exists(f'{invokeai_root}/venv'):
    !virtualenv venv
    !source venv/bin/activate -p; pip install TensorRT
    !source venv/bin/activate; pip install InvokeAI[xformers] 
    !source venv/bin/activate; invokeai-configure --yes
!source venv/bin/activate -p; pip check

In [ ]:
#@title ## `huggingface login`
from os import environ
import sys, os
!source venv/bin/activate; pip install huggingface_hub
!source venv/bin/activate; huggingface-cli login --token=$huggingface_token

# `Start NGROK, open link after launching InvokeAI`

In [ ]:
from IPython.display import display 

# %pip install pyngrok
from pyngrok import ngrok

ngrok.kill()
!ngrok config add-authtoken $ngrok_token
ngrok.set_auth_token(token= f'{ngrok_token}')
public_url = ngrok.connect(9090).public_url


# show the link
display(Markdown("# WEBUI \n------\n`InvokeAI Public URL :`\n"))
print(f'{public_url}')
display(Markdown("\n\n\n------\n\n ` click once running` "))



In [ ]:
#@title # ` start it up `
os.chdir(invokeai_root)
!source venv/bin/activate; invokeai --no-nsfw_checker --no-safety_checker --web  --xformers --host 0.0.0.0